In [1]:
import sys
sys.path

['c:\\Users\\Shahir\\Documents\\Code Repositories\\cs330-final-project-2022\\scripts',
 'C:\\Users\\Shahir\\Documents\\Code Repositories\\6857-final-project',
 'C:\\Users\\Shahir\\Documents\\Code Repositories\\sdn-umw-app',
 'C:\\Users\\Shahir\\Documents\\Code Repositories\\WiFreshApp',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\python310.zip',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\DLLs',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\lib',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project',
 '',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\lib\\site-packages',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\lib\\site-packages\\win32',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\Shahir\\anaconda3\\envs\\cs330-final-project\\lib\\site-packages\\Pythonwin']

In [2]:
import cs330_project

ModuleNotFoundError: No module named 'cs330_project'

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

from cs330_project.datasets import RawDataset
from cs330_project.training import train_one_epoch

n, c, h, w = 10, 3, 64, 64
x_data = np.random.rand(n, c, h, w)
y_data = np.random.randint(0, 10, n)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20)
dataset_train = RawDataset(x_train, y_train)
dataset_test = RawDataset(x_test, y_test)

In [ ]:
from cs330_project.temp2.mae_st.models_mae import mae_vit_base_patch16

model = mae_vit_base_patch16()